In [100]:
import sys
sys.path.append('../..')
from src import WindSpeedUnit, ForecastAPI, ForecastCurrent, HistoricalAPI, HistoricalDaily, HistoricalHourly
import src
import pandas as pd
import datetime as dt

f = HistoricalAPI()
f.latitude, f.longitude = (53.98352, -6.39139) #src.GEO_COORDINATES.DUNDALK_IT.value
f.wind_speed_unit = WindSpeedUnit.METERS_PER_SECOND
f.daily = [HistoricalDaily.TEMPERATURE_2M_MEAN, HistoricalDaily.WIND_SPEED_10M_MAX]
f.hourly = [HistoricalHourly.TEMPERATURE_2M, HistoricalHourly.WIND_SPEED_10M]
f.start_date = dt.datetime(2020, 1, 1)
f.end_date = dt.datetime(2020, 1, 2)
f.cell_selection = src.enums.CellSelection.NEAREST

response = f.request()
print(f.build_url())
print(response)

Remaining: {'daily': 9985.0, 'hourly': 4995.0, 'minutely': 600}
API counts: 1.0
https://archive-api.open-meteo.com/v1/archive?cell_selection=nearest&daily=temperature_2m_mean,wind_speed_10m_max&end_date=2020-01-02&hourly=temperature_2m,wind_speed_10m&latitude=53.98352&longitude=-6.39139&start_date=2020-01-01&wind_speed_unit=ms
{'latitude': 53.954304, 'longitude': -6.4410095, 'generationtime_ms': 0.07081031799316406, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 15.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C', 'wind_speed_10m': 'm/s'}, 'hourly': {'time': ['2020-01-01T00:00', '2020-01-01T01:00', '2020-01-01T02:00', '2020-01-01T03:00', '2020-01-01T04:00', '2020-01-01T05:00', '2020-01-01T06:00', '2020-01-01T07:00', '2020-01-01T08:00', '2020-01-01T09:00', '2020-01-01T10:00', '2020-01-01T11:00', '2020-01-01T12:00', '2020-01-01T13:00', '2020-01-01T14:00', '2020-01-01T15:00', '2020-01-01T16:00', '2020-01-01T17:00', '2020-01-01T18:00',

In [135]:
'https://archive-api.open-meteo.com/v1/archive?cell_selection=nearest&daily=temperature_2m_mean,wind_speed_10m_max&end_date=2020-01-02&hourly=temperature_2m,wind_speed_10m&latitude=50.98352&longitude=6.39139&latitude=53.98352&longitude=-6.39139&start_date=2020-01-01&wind_speed_unit=ms'

'https://archive-api.open-meteo.com/v1/archive?cell_selection=nearest&daily=temperature_2m_mean,wind_speed_10m_max&end_date=2020-01-02&hourly=temperature_2m,wind_speed_10m&latitude=50.98352&longitude=6.39139&latitude=53.98352&longitude=-6.39139&start_date=2020-01-01&wind_speed_unit=ms'

In [ ]:
url = 'https://archive-api.open-meteo.com/v1/archive?latitude=52.0&longitude=12.0&latitude=53.98352&longitude=-6.39139&start_date=2025-02-01&end_date=2025-02-19&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,boundary_layer_height,wet_bulb_temperature_2m,total_column_integrated_water_vapour,is_day,sunshine_duration,albedo,snow_depth_water_equivalent&daily=weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,daylight_duration,sunshine_duration,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration&wind_speed_unit=ms&timezone=auto'
src.weather.utils.ApiCounter.calculate_call_weight_from_url(url) #3718.842 is the answer. It's wrong though.
# src.GEO_COORDINATES.DUNDALK_IT.value

# pd.read_pickle('historical.pkl').longitude[0]


7.328571428571429

In [102]:
import datetime as dt

parsedParams = src.weather.utils.ApiCounter.parse_url_params(url)
startDate = dt.datetime.strptime(parsedParams['start_date'], '%Y-%m-%d')#.strftime('%Y-%m-%d %H:%M:%S')
endDate = dt.datetime.strptime(parsedParams['end_date'], '%Y-%m-%d')#.strftime('%Y-%m-%d %H:%M:%S')
parsedParams['start_date'] = startDate.strftime('%Y-%m-%d %H:%M:%S')
parsedParams['end_date'] = endDate.strftime('%Y-%m-%d %H:%M:%S')

queryDaily = pd.DataFrame(data = pd.date_range(start=startDate, end=endDate, freq='D'), columns=['time'])
queryDaily = queryDaily.merge(pd.DataFrame(data = parsedParams['daily'],columns = ['parameter']), how='cross')
queryDaily = queryDaily.merge(
    pd.DataFrame(dict(latitude = parsedParams['latitude'], longitude = parsedParams['longitude'])),
    how='cross')
queryDaily.time = queryDaily.time.dt.strftime('%Y-%m-%d %H:%M:%S')


queryHourly = pd.DataFrame(data = pd.date_range(start=startDate,end=endDate, freq='h'), columns=['time'])
queryHourly = queryHourly.merge(pd.DataFrame(data = parsedParams['hourly'],columns = ['parameter']), how='cross')
queryHourly = queryHourly.merge(
    pd.DataFrame(dict(latitude = parsedParams['latitude'], longitude = parsedParams['longitude'])),
    how='cross')
queryHourly.time = queryHourly.time.dt.strftime('%Y-%m-%d %H:%M:%S')

#ensure dtypes
queryDaily['time'] = queryDaily['time'].astype('str')
queryDaily['parameter'] = queryDaily['parameter'].astype('str')
queryDaily['latitude'] = queryDaily['latitude'].astype('float')
queryDaily['longitude'] = queryDaily['longitude'].astype('float')

queryHourly['time'] = queryHourly['time'].astype('str')
queryHourly['parameter'] = queryHourly['parameter'].astype('str')
queryHourly['latitude'] = queryHourly['latitude'].astype('float')
queryHourly['longitude'] = queryHourly['longitude'].astype('float')

# make stging tables
conn = f._conn
cursor = conn.cursor()

hourlyStageCommand = '''
CREATE TABLE IF NOT EXISTS hourly_historical_weather_staging_table(
    time TEXT,
    parameter TEXT,
    latitude REAL,
    longitude REAL
);
'''
cursor.execute(hourlyStageCommand)
conn.commit()

dailyStageCommand = '''
CREATE TABLE IF NOT EXISTS daily_historical_weather_staging_table(
    time TEXT,
    parameter TEXT,
    latitude REAL,
    longitude REAL
);
'''
cursor.execute( dailyStageCommand)
conn.commit()

#insert data
queryDaily.to_sql('daily_historical_weather_staging_table', conn, if_exists='replace', index=False)
queryHourly.to_sql('hourly_historical_weather_staging_table', conn, if_exists='replace', index=False)

32042

In [121]:
# create a command that joins the staging table (request table) with the historical data and then pull it out
EPSILON = 1E-12
# dailySearchCommand = '''
# SELECT * FROM (
#         SELECT * FROM daily_historical_weather_data
#         WHERE time BETWEEN ? AND ?
#     ) AS DAILY
# LEFT JOIN daily_historical_weather_staging_table
# ON daily_historical_weather_staging_table.time = DAILY.time
# AND daily_historical_weather_staging_table.parameter = DAILY.parameter
# AND daily_historical_weather_staging_table.latitude = DAILY.latitude
# AND daily_historical_weather_staging_table.longitude = DAILY.longitude
# '''

dailySearchCommand = f'''
SELECT
    COALESCE(DAILY.time, daily_historical_weather_staging_table.time) AS time,
    COALESCE(DAILY.parameter, daily_historical_weather_staging_table.parameter) AS parameter,
    COALESCE(DAILY.latitude, daily_historical_weather_staging_table.latitude) AS latitude,
    COALESCE(DAILY.longitude, daily_historical_weather_staging_table.longitude) AS longitude,
    DAILY.value
FROM (
    SELECT time, parameter, latitude, longitude, value
    FROM daily_historical_weather_data
    WHERE time BETWEEN ? AND ?
) AS DAILY
RIGHT JOIN daily_historical_weather_staging_table
ON daily_historical_weather_staging_table.time = DAILY.time
AND daily_historical_weather_staging_table.parameter = DAILY.parameter
AND ABS(daily_historical_weather_staging_table.latitude - DAILY.latitude) < {EPSILON}
AND ABS(daily_historical_weather_staging_table.longitude - DAILY.longitude) < {EPSILON}
'''
responseDaily = pd.read_sql_query(dailySearchCommand, conn, params=[parsedParams['start_date'], parsedParams['end_date']])


hourlySearchCommand = f'''
SELECT 
    COALESCE(HOURLY.time, hourly_historical_weather_staging_table.time) AS time,
    COALESCE(HOURLY.parameter, hourly_historical_weather_staging_table.parameter) AS parameter,
    COALESCE(HOURLY.latitude, hourly_historical_weather_staging_table.latitude) AS latitude,
    COALESCE(HOURLY.longitude, hourly_historical_weather_staging_table.longitude) AS longitude,
    HOURLY.value
FROM (
    SELECT time, parameter, latitude, longitude, value
    FROM hourly_historical_weather_data
    WHERE time BETWEEN ? AND ?
) AS HOURLY
RIGHT JOIN hourly_historical_weather_staging_table
ON hourly_historical_weather_staging_table.time = HOURLY.time
AND hourly_historical_weather_staging_table.parameter = HOURLY.parameter
AND ABS(hourly_historical_weather_staging_table.latitude - HOURLY.latitude) < {EPSILON}
AND ABS(hourly_historical_weather_staging_table.longitude - HOURLY.longitude) < {EPSILON}
'''

responseHourly = pd.read_sql_query(hourlySearchCommand, conn, params=[parsedParams['start_date'], parsedParams['end_date']])

In [ ]:
# foundHourly.loc[lambda s:s.isna().any(axis=1)]
foundHourly = responseHourly.copy().loc[lambda s:s.value.notna()]
foundDaily = responseDaily.copy().loc[lambda s:s.value.notna()]
notFoundHourly = responseHourly.copy().loc[lambda s:s.value.isna()]
notFoundDaily = responseDaily.copy().loc[lambda s:s.value.isna()]

# now what? make a serparate requests for the missing data

In [152]:
notFoundHourly = notFoundHourly.assign(geo = lambda s: s.latitude.astype('str') + ', ' + s.longitude.astype('str'))
notFoundHourly.sort_values(by = 'geo parameter time'.split(), inplace=True, ignore_index=True)
notFoundHourly['timeObj'] = pd.to_datetime(notFoundHourly.time)

for key,grp in notFoundHourly.groupby('geo parameter'.split()):
    break
((grp.timeObj.shift(-1)-grp.timeObj).fillna(pd.Timedelta(hours=1)).dt.total_seconds()/3600).sum()

np.float64(433.0)